In [ ]:
# The aim of this module is to use the Ben Graham model to score the companies

# File name convention
# R_ = raw extract 
# E_ = Entry files into enrichment layer
# C_ = enriched layer with calculation
# I = insights layer, designed for model baselines
# V_ = validation


In [1]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn'

I_FLAT = pd.read_pickle ('/Users/joezhou/Downloads/Investment/1 data/C_TICKER_FLAT.pkl') 


In [2]:
# from datetime import datetime as dt
# dte = dt.now().strftime('%Y%m%d')
# fname = "/Users/joezhou/Downloads/Investment/TEST_BG_VARS{}.xlsx".format(dte)
# I_FLAT.to_excel(fname)

In [26]:
# Selection using ENTervative investment approach
# large company with track record, with good long standing financials

BG_CONS = I_FLAT

BG_CONS['CALC_BG_CONS_2_SALES'] = np.where(BG_CONS['Total Revenue'] > 1000000000 , 1, 0)
BG_CONS['CALC_BG_CONS_3_CURR_RATIO'] = np.where(BG_CONS['Total Current Assets']/BG_CONS['Total Current Liabilities'] > 2 , 1, 0)
BG_CONS['CALC_BG_CONS_4_DIV'] = np.where((BG_CONS['CALC_DIV_5YR_AMT'] ) > 0, 1, 0)
BG_CONS['CALC_BG_CONS_4_DIV_GTH'] = np.where((BG_CONS['CALC_DIV_3YR_AMT']/3) <  BG_CONS['CALC_DIV_1YR_AMT'] , 1, 0)

BG_CONS['CALC_BG_CONS_5_ERN'] = np.where(BG_CONS['Retained Earnings'] > 0 , 1, 0)
BG_CONS['CALC_BG_CONS_6_ERN_GTH'] = np.where(BG_CONS['earningsQuarterlyGrowth'] > .03 , 1, 0)

# 7. Market cap < (asset - liability) * 1.5
BG_CONS['CALC_BG_CONS_7_MKT_CAP'] = np.where(BG_CONS['marketCap'] < (BG_CONS['Total Assets'] - BG_CONS['Total Liab'] )*1.5 , 1, 0)
BG_CONS['CALC_BG_CONS_8_PE'] = np.where(BG_CONS['Last_Close_price']/(BG_CONS['Net Income']/BG_CONS['sharesOutstanding']) < 15 , 1, 0)

BG_CONS['CALC_BG_CONS_BG_SCORE'] = BG_CONS['CALC_BG_CONS_2_SALES'] + BG_CONS['CALC_BG_CONS_3_CURR_RATIO'] + BG_CONS['CALC_BG_CONS_4_DIV']+ BG_CONS['CALC_BG_CONS_5_ERN']+ BG_CONS['CALC_BG_CONS_6_ERN_GTH']+ BG_CONS['CALC_BG_CONS_7_MKT_CAP']

# only keep variables used in and output from BG mode
BG_CONS_VARS = BG_CONS[['TickName','Total Revenue','Total Current Assets','Total Current Liabilities','CALC_DIV_5YR_AMT','CALC_DIV_3YR_AMT','CALC_DIV_1YR_AMT','Retained Earnings','earningsQuarterlyGrowth',
'marketCap','Total Assets','Total Liab','trailingPE','Last_Close_price','Net Income','sharesOutstanding',
'CALC_BG_CONS_2_SALES','CALC_BG_CONS_3_CURR_RATIO','CALC_BG_CONS_4_DIV','CALC_BG_CONS_4_DIV_GTH','CALC_BG_CONS_5_ERN',
'CALC_BG_CONS_6_ERN_GTH','CALC_BG_CONS_7_MKT_CAP','CALC_BG_CONS_8_PE','CALC_BG_CONS_BG_SCORE'
]]

BG_CONS_SUM = BG_CONS.groupby(['CALC_BG_CONS_BG_SCORE'])['TickName'].count().reset_index()
BG_CONS_SUM.head(10)



,CALC_BG_CONS_BG_SCORE,TickName
0,0,15
1,1,60
2,2,108
3,3,152
4,4,125
5,5,39
6,6,1


In [27]:
# Selection using enterprising investment approach
# large company positioned for growth
# 

BG_ENT = I_FLAT

BG_ENT['CALC_BG_ENT_2_SALES'] = np.where(BG_ENT['Total Revenue'] > 0 , 1, 0)
BG_ENT['CALC_BG_ENT_3_CURR_RATIO'] = np.where(BG_CONS['Total Current Assets']/BG_CONS['Total Current Liabilities'] > 1.5 , 1, 0)
BG_ENT['CALC_BG_ENT_4_DIV'] = np.where((BG_CONS['CALC_DIV_1YR_AMT'] ) > 0, 1, 0)

BG_ENT['CALC_BG_ENT_5_ERN'] = np.where(BG_ENT['grossProfits'] > 0 , 1, 0)
BG_ENT['CALC_BG_ENT_6_ERN_GTH'] = np.where(BG_ENT['earningsGrowth'] > .02 , 1, 0)

# 7. Market cap < (asset - liability) * 1.5

BG_ENT['CALC_BG_ENT_7_MKT_CAP'] = np.where(BG_CONS['marketCap'] < (BG_CONS['Total Assets'] - BG_CONS['Total Liab'] )*1.2 , 1, 0)
BG_ENT['CALC_BG_ENT_8_PE'] = np.where(BG_CONS['Last_Close_price']/(BG_CONS['Net Income']/BG_CONS['sharesOutstanding']) < 20 , 1, 0)

BG_ENT['CALC_BG_ENT_BG_SCORE'] = BG_ENT['CALC_BG_ENT_2_SALES'] + BG_ENT['CALC_BG_ENT_3_CURR_RATIO'] + BG_ENT['CALC_BG_ENT_4_DIV']+ BG_ENT['CALC_BG_ENT_5_ERN']+ BG_ENT['CALC_BG_ENT_6_ERN_GTH']+ BG_ENT['CALC_BG_ENT_7_MKT_CAP']


# only keep variables used in and output from BG model
BG_ENT_VARS = BG_ENT[['TickName','CALC_BG_ENT_2_SALES','CALC_BG_ENT_3_CURR_RATIO','CALC_BG_ENT_4_DIV','CALC_BG_ENT_5_ERN',
'CALC_BG_ENT_6_ERN_GTH','CALC_BG_ENT_7_MKT_CAP','CALC_BG_ENT_8_PE','CALC_BG_ENT_BG_SCORE'
]]

BG_ENT_SUM = BG_ENT.groupby(['CALC_BG_ENT_BG_SCORE'])['TickName'].count().reset_index()
BG_ENT_SUM.head(10)


,CALC_BG_ENT_BG_SCORE,TickName
0,0,1
1,1,17
2,2,33
3,3,116
4,4,179
5,5,126
6,6,28


In [28]:
BG_MODEL = BG_CONS_VARS.merge(BG_ENT_VARS,how='left',on='TickName')
BG_MODEL.to_pickle("/Users/joezhou/Downloads/Investment/1 data/C_BEN_G_MODEL.pkl")


from datetime import datetime as dt
dte = dt.now().strftime('%Y%m%d')
fname = "/Users/joezhou/Downloads/Investment/BEN_G_OUTPUTS{}.xlsx".format(dte)
BG_MODEL.to_excel(fname)

In [29]:
I_PRICE = pd.read_pickle ('/Users/joezhou/Downloads/Investment/1 data/E_ALL_Prices.pkl') 


BG_MODEL_PRICE = BG_MODEL.merge(I_PRICE,how='left',on='TickName')


from datetime import datetime as dt
dte = dt.now().strftime('%Y%m%d')
fname = "/Users/joezhou/Downloads/Investment/BEN_G_PRICE_OUTPUTS{}.xlsx".format(dte)
BG_MODEL_PRICE.to_excel(fname)
